In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
ph = pd.read_csv('pasthistory.csv')
ph.head()

,Unnamed: 0,pasthistoryid,patientunitstayid,pasthistoryoffset,pasthistoryenteredoffset,pasthistorynotetype,pasthistorypath,pasthistoryvalue,pasthistoryvaluetext
0,1,2919413,435329,311,322,Initial Consultation/Other,notes/Progress Notes/Past History/Organ System...,pulmonary embolism - date unknown,pulmonary embolism - date unknown
1,2,3263540,445612,95,108,Initial Consultation/Other,notes/Progress Notes/Past History/Organ System...,s/p AVR,s/p AVR
2,3,3263539,445612,95,108,Initial Consultation/Other,notes/Progress Notes/Past History/Organ System...,AR,AR
3,4,3512180,449958,413,422,Initial Consultation/Other,notes/Progress Notes/Past History/Organ System...,chronic kidney stones,chronic kidney stones
4,5,2524609,451540,70,95,Initial Consultation/Other,notes/Progress Notes/Past History/Organ System...,TIA(s) - remote,TIA(s) - remote


In [3]:
pts= list(ph['patientunitstayid'].drop_duplicates())
len(pts)

29626

In [4]:
mapping = pd.read_csv('ph_mapping.csv')
mapping

,Pasthistorypath,Paper_a,Paper_b,Charlson,score,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,X
0,notes/Progress Notes/Past History/Organ System...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,?
1,notes/Progress Notes/Past History/Organ System...,Coronary Atherosclerosis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Acute cerebrovascular disease,X
2,notes/Progress Notes/Past History/Organ System...,Coronary Atherosclerosis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anemia,X
3,notes/Progress Notes/Past History/Organ System...,Coronary Atherosclerosis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Atrial fibrillation,X
4,notes/Progress Notes/Past History/Organ System...,Coronary Atherosclerosis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Blood Malignancy,?
5,notes/Progress Notes/Past History/Organ System...,Coronary Atherosclerosis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chronic kidney disease,?
6,notes/Progress Notes/Past History/Organ System...,Coronary Atherosclerosis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chronic obstructive pulmonary disease and bron...,X
7,notes/Progress Notes/Past History/Organ System...,Atrial fibrillation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coronary atherosclerosis,?
8,notes/Progress Notes/Past History/Organ System...,Atrial fibrillation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Diabetes mellitus,X
9,notes/Progress Notes/Past History/Organ System...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Heart valve disorders,X


In [5]:
mapping = mapping[pd.notna(mapping['Paper_a'])]
pathlist = list(mapping['Pasthistorypath'])
len(mapping)

150

In [6]:
mappingb = mapping[pd.notna(mapping['Paper_b'])]
len(mappingb)

8

In [7]:
mappingc = mapping[pd.notna(mapping['Charlson'])]
len(mappingc)

110

In [8]:
comorbidities = ['Acute Cerebrovascular Disease',
'Anemia',
'Atrial fibrillation',
'Blood Malignancy',
'Chronic Kidney Disease',
'Chronic obstructive pulmonary disease and bronchiectasis',
'Coronary Atherosclerosis',
'Diabetes mellitus',
'Heart valve disorders',
'Hypertension',
'Neoplasms',
'MI',
'Congestive Heart Failure',
'Healthy',
 'Unknown'                
]

In [9]:
charlson = ['priormi', 'chf', 'pvd', 'diabetes', 'mld', 'sld', 'pud', 
            'cancer', 'leukemia', 'lymphoma', 'met', 'aids', 'dementia',
           'cvd', 'cpd', 'renaldisease', 'rheum', 'charlson_score']

cvals =  [1, 1, 1, 1, 3, 3, 1, 2, 2, 2, 6, 6, 1, 1, 1, 2, 1]

In [10]:
charlsondf = pd.DataFrame(index = pts, columns = charlson)


In [11]:
outs = pd.DataFrame(index = pts, columns = comorbidities)

In [12]:
for row in np.arange(len(ph)):   

    name = mapping[mapping['Pasthistorypath'] == ph['pasthistorypath'][row]]['Paper_a']
    
    if(len(name) > 0):
        outs[name.iloc[0]][ph['patientunitstayid'][row]] = 1
    
    name2 = mappingb[mappingb['Pasthistorypath'] == ph['pasthistorypath'][row]]['Paper_b']
    
    if(len(name2) > 0):
        outs[name2.iloc[0]][ph['patientunitstayid'][row]] = 1
        
    name3 = mappingc[mappingc['Pasthistorypath'] == ph['pasthistorypath'][row]]['Charlson']
    
    if(len(name3) > 0):
        charlsondf[name3.iloc[0]][ph['patientunitstayid'][row]] = 1
    
    if(row%1000 == 0):
        print(row/len(ph))

0.0
0.00644118234343096
0.01288236468686192
0.01932354703029288
0.02576472937372384
0.0322059117171548
0.03864709406058576
0.04508827640401672
0.05152945874744768
0.05797064109087864
0.0644118234343096
0.07085300577774056
0.07729418812117152
0.08373537046460248
0.09017655280803344
0.0966177351514644
0.10305891749489536
0.10950009983832633
0.11594128218175728
0.12238246452518825
0.1288236468686192
0.13526482921205016
0.14170601155548113
0.1481471938989121
0.15458837624234303
0.161029558585774
0.16747074092920497
0.17391192327263594
0.18035310561606688
0.18679428795949785
0.1932354703029288
0.19967665264635975
0.20611783498979072
0.2125590173332217
0.21900019967665266
0.2254413820200836
0.23188256436351457
0.23832374670694553
0.2447649290503765
0.25120611139380744
0.2576472937372384
0.2640884760806694
0.2705296584241003
0.2769708407675313
0.28341202311096225
0.2898532054543932
0.2962943877978242
0.30273557014125513
0.30917675248468607
0.31561793482811706
0.322059117171548
0.3285002995149

In [13]:
(outs.notna().sum().round(4) * 100)/len(pts)

Acute Cerebrovascular Disease                               11.061230
Anemia                                                       0.678458
Atrial fibrillation                                         13.872949
Blood Malignancy                                             1.795720
Chronic Kidney Disease                                      14.487275
Chronic obstructive pulmonary disease and bronchiectasis    16.056842
Coronary Atherosclerosis                                    25.848916
Diabetes mellitus                                           31.171944
Heart valve disorders                                        8.863836
Hypertension                                                61.375143
Neoplasms                                                   11.894957
MI                                                          14.007966
Congestive Heart Failure                                    20.191723
Healthy                                                      4.242895
Unknown             

In [14]:
charlsondf.fillna(0, inplace=True)

In [15]:
charlsondf['patientunitstayid'] = charlsondf.index
charlsondf = charlsondf.reset_index()

In [16]:
charlsondf.head()

,index,priormi,chf,pvd,diabetes,mld,sld,pud,cancer,leukemia,lymphoma,met,aids,dementia,cvd,cpd,renaldisease,rheum,charlson_score,patientunitstayid
0,435329,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,435329
1,445612,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,445612
2,449958,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,449958
3,451540,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,451540
4,454515,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,454515


In [17]:
for row in np.arange(len(charlsondf)):
    charlsondf['charlson_score'][row] = 0
    for i, var in enumerate(charlson[:-1]):
        charlsondf['charlson_score'][row] += cvals[i] * charlsondf[var][row]
        
    if(row%1000 == 0):
        print(row/len(ph))

0.0
0.00644118234343096
0.01288236468686192
0.01932354703029288
0.02576472937372384
0.0322059117171548
0.03864709406058576
0.04508827640401672
0.05152945874744768
0.05797064109087864
0.0644118234343096
0.07085300577774056
0.07729418812117152
0.08373537046460248
0.09017655280803344
0.0966177351514644
0.10305891749489536
0.10950009983832633
0.11594128218175728
0.12238246452518825
0.1288236468686192
0.13526482921205016
0.14170601155548113
0.1481471938989121
0.15458837624234303
0.161029558585774
0.16747074092920497
0.17391192327263594
0.18035310561606688
0.18679428795949785


In [18]:
charlsondf['charlson_score'].value_counts()/len(charlsondf)

0     0.342571
1     0.251502
2     0.172180
3     0.108655
4     0.062546
5     0.030851
6     0.012658
8     0.007190
7     0.004658
9     0.003848
10    0.002059
11    0.000945
12    0.000203
13    0.000101
14    0.000034
Name: charlson_score, dtype: float64

In [19]:
outs.fillna(0, inplace=True)
outs['patientunitstayid'] = outs.index

In [20]:
charlsondf = charlsondf.reset_index()

In [21]:
output = pd.merge(outs, charlsondf[['patientunitstayid','charlson_score']], on = 'patientunitstayid', how = 'inner')

In [22]:
output.to_csv('comorbidities_may5th.csv')

In [23]:
charlsondf.to_csv('charlson_comorbidities_may6th.csv')

In [24]:
output

,Acute Cerebrovascular Disease,Anemia,Atrial fibrillation,Blood Malignancy,Chronic Kidney Disease,Chronic obstructive pulmonary disease and bronchiectasis,Coronary Atherosclerosis,Diabetes mellitus,Heart valve disorders,Hypertension,Neoplasms,MI,Congestive Heart Failure,Healthy,Unknown,patientunitstayid,charlson_score
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,435329,0
1,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,445612,1
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,449958,2
3,1,0,0,0,0,1,1,0,0,1,0,1,1,0,0,451540,4
4,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,454515,2
5,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,458131,3
6,0,0,0,0,0,0,1,1,0,1,0,1,1,0,0,473844,3
7,0,0,0,0,1,0,1,1,0,1,0,0,1,0,0,478650,4
8,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,501038,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,507902,0
